## एम्बेडिंग्स

हमारे पिछले उदाहरण में, हमने उच्च-आयामी बैग-ऑफ-वर्ड्स वेक्टर पर काम किया था, जिसकी लंबाई `vocab_size` थी, और हम स्पष्ट रूप से निम्न-आयामी स्थानिक प्रतिनिधित्व वेक्टर को विरल वन-हॉट प्रतिनिधित्व में परिवर्तित कर रहे थे। यह वन-हॉट प्रतिनिधित्व मेमोरी-कुशल नहीं है, इसके अलावा, प्रत्येक शब्द को एक-दूसरे से स्वतंत्र रूप से माना जाता है, यानी वन-हॉट एन्कोडेड वेक्टर शब्दों के बीच किसी भी अर्थपूर्ण समानता को व्यक्त नहीं करते हैं।

इस इकाई में, हम **News AG** डेटासेट का अन्वेषण जारी रखेंगे। शुरू करने के लिए, आइए डेटा लोड करें और पिछले नोटबुक से कुछ परिभाषाएँ प्राप्त करें।


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## एम्बेडिंग क्या है?

**एम्बेडिंग** का विचार यह है कि शब्दों को निम्न-आयामी घने वेक्टरों द्वारा दर्शाया जाए, जो किसी तरह शब्द के अर्थ को प्रतिबिंबित करते हैं। हम बाद में चर्चा करेंगे कि सार्थक शब्द एम्बेडिंग कैसे बनाई जाए, लेकिन फिलहाल, एम्बेडिंग को शब्द वेक्टर की आयाम संख्या को कम करने के तरीके के रूप में सोचें।

इस प्रकार, एम्बेडिंग लेयर एक शब्द को इनपुट के रूप में लेगी और निर्दिष्ट `embedding_size` का आउटपुट वेक्टर उत्पन्न करेगी। एक अर्थ में, यह `Linear` लेयर के समान है, लेकिन एक-हॉट एनकोडेड वेक्टर लेने के बजाय, यह एक शब्द संख्या को इनपुट के रूप में ले सकेगी।

हमारे नेटवर्क में एम्बेडिंग लेयर को पहली लेयर के रूप में उपयोग करके, हम बैग-ऑफ-वर्ड्स से **एम्बेडिंग बैग** मॉडल में स्विच कर सकते हैं, जहां हम पहले अपने टेक्स्ट के प्रत्येक शब्द को संबंधित एम्बेडिंग में बदलते हैं, और फिर उन सभी एम्बेडिंग पर कुछ समग्र फ़ंक्शन की गणना करते हैं, जैसे `sum`, `average` या `max`।

![पांच अनुक्रम शब्दों के लिए एम्बेडिंग क्लासिफायर दिखाने वाली छवि।](../../../../../lessons/5-NLP/14-Embeddings/images/embedding-classifier-example.png)

हमारा क्लासिफायर न्यूरल नेटवर्क एम्बेडिंग लेयर से शुरू होगा, फिर एग्रीगेशन लेयर, और उसके ऊपर एक लीनियर क्लासिफायर:


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### चर अनुक्रम आकार से निपटना

इस आर्किटेक्चर के परिणामस्वरूप, हमारे नेटवर्क के लिए मिनीबैच को एक विशेष तरीके से बनाना होगा। पिछले यूनिट में, जब बैग-ऑफ-वर्ड्स (BoW) का उपयोग कर रहे थे, तो मिनीबैच में सभी BoW टेंसर का आकार `vocab_size` के बराबर होता था, चाहे हमारे टेक्स्ट अनुक्रम की वास्तविक लंबाई कुछ भी हो। जब हम वर्ड एम्बेडिंग्स पर जाते हैं, तो प्रत्येक टेक्स्ट सैंपल में शब्दों की संख्या अलग-अलग हो सकती है, और इन सैंपल्स को मिनीबैच में मिलाने के दौरान हमें कुछ पैडिंग लागू करनी होगी।

यह `collate_fn` फ़ंक्शन को डेटा स्रोत में प्रदान करने की उसी तकनीक का उपयोग करके किया जा सकता है:


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### एम्बेडिंग क्लासिफायर का प्रशिक्षण

अब जब हमने सही डाटालोडर परिभाषित कर लिया है, तो हम पिछले यूनिट में परिभाषित प्रशिक्षण फ़ंक्शन का उपयोग करके मॉडल को प्रशिक्षित कर सकते हैं:


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

> **नोट**: हम यहां केवल 25k रिकॉर्ड्स (एक पूर्ण युग से कम) के लिए प्रशिक्षण कर रहे हैं समय बचाने के लिए, लेकिन आप प्रशिक्षण जारी रख सकते हैं, कई युगों के लिए प्रशिक्षण के लिए एक फ़ंक्शन लिख सकते हैं, और उच्च सटीकता प्राप्त करने के लिए लर्निंग रेट पैरामीटर के साथ प्रयोग कर सकते हैं। आपको लगभग 90% सटीकता तक पहुंचने में सक्षम होना चाहिए।


### EmbeddingBag लेयर और चर लंबाई अनुक्रम प्रतिनिधित्व

पिछली संरचना में, हमें सभी अनुक्रमों को एक ही लंबाई तक बढ़ाना पड़ता था ताकि उन्हें मिनीबैच में फिट किया जा सके। यह चर लंबाई अनुक्रमों को प्रस्तुत करने का सबसे प्रभावी तरीका नहीं है - एक अन्य दृष्टिकोण **ऑफसेट** वेक्टर का उपयोग करना होगा, जो एक बड़े वेक्टर में संग्रहीत सभी अनुक्रमों के ऑफसेट को रखेगा।

![ऑफसेट अनुक्रम प्रतिनिधित्व दिखाने वाली छवि](../../../../../lessons/5-NLP/14-Embeddings/images/offset-sequence-representation.png)

> **Note**: ऊपर दी गई तस्वीर में, हमने अक्षरों के अनुक्रम को दिखाया है, लेकिन हमारे उदाहरण में हम शब्दों के अनुक्रमों के साथ काम कर रहे हैं। हालांकि, ऑफसेट वेक्टर के साथ अनुक्रमों को प्रस्तुत करने का सामान्य सिद्धांत वही रहता है।

ऑफसेट प्रतिनिधित्व के साथ काम करने के लिए, हम [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html) लेयर का उपयोग करते हैं। यह `Embedding` के समान है, लेकिन यह सामग्री वेक्टर और ऑफसेट वेक्टर को इनपुट के रूप में लेता है, और इसमें औसत लेयर भी शामिल होती है, जो `mean`, `sum` या `max` हो सकती है।

यहाँ एक संशोधित नेटवर्क है जो `EmbeddingBag` का उपयोग करता है:


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

डेटासेट को प्रशिक्षण के लिए तैयार करने के लिए, हमें एक रूपांतरण फ़ंक्शन प्रदान करना होगा जो ऑफ़सेट वेक्टर तैयार करेगा:


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

ध्यान दें कि सभी पिछले उदाहरणों के विपरीत, हमारा नेटवर्क अब दो पैरामीटर स्वीकार करता है: डेटा वेक्टर और ऑफसेट वेक्टर, जो अलग-अलग आकार के होते हैं। इसी तरह, हमारा डेटा लोडर भी हमें 2 के बजाय 3 मान प्रदान करता है: टेक्स्ट और ऑफसेट वेक्टर दोनों को फीचर्स के रूप में प्रदान किया जाता है। इसलिए, हमें अपने प्रशिक्षण फ़ंक्शन को थोड़ा समायोजित करने की आवश्यकता है ताकि इसका ध्यान रखा जा सके:


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## सेमांटिक एम्बेडिंग्स: वर्ड2वेक

पिछले उदाहरण में, मॉडल एम्बेडिंग लेयर ने शब्दों को वेक्टर प्रतिनिधित्व में मैप करना सीखा, लेकिन इस प्रतिनिधित्व में ज्यादा सेमांटिक अर्थ नहीं था। यह अच्छा होगा कि ऐसा वेक्टर प्रतिनिधित्व सीखा जाए, जिसमें समान शब्द या पर्यायवाची शब्द ऐसे वेक्टर से मेल खाएं जो किसी वेक्टर दूरी (जैसे, यूक्लिडियन दूरी) के संदर्भ में एक-दूसरे के करीब हों।

इसके लिए, हमें अपने एम्बेडिंग मॉडल को एक बड़े टेक्स्ट संग्रह पर एक विशेष तरीके से प्री-ट्रेन करना होगा। सेमांटिक एम्बेडिंग्स को ट्रेन करने के शुरुआती तरीकों में से एक को [वर्ड2वेक](https://en.wikipedia.org/wiki/Word2vec) कहा जाता है। यह शब्दों के वितरित प्रतिनिधित्व को उत्पन्न करने के लिए दो मुख्य आर्किटेक्चर पर आधारित है:

 - **कंटीन्युअस बैग-ऑफ-वर्ड्स** (CBoW) — इस आर्किटेक्चर में, हम मॉडल को आस-पास के संदर्भ से एक शब्द की भविष्यवाणी करने के लिए ट्रेन करते हैं। दिए गए ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$ में, मॉडल का लक्ष्य $(W_{-2},W_{-1},W_1,W_2)$ से $W_0$ की भविष्यवाणी करना है।
 - **कंटीन्युअस स्किप-ग्राम** CBoW के विपरीत है। मॉडल संदर्भ शब्दों की आस-पास की विंडो का उपयोग करके वर्तमान शब्द की भविष्यवाणी करता है।

CBoW तेज है, जबकि स्किप-ग्राम धीमा है, लेकिन यह कम बार उपयोग होने वाले शब्दों का बेहतर प्रतिनिधित्व करता है।

![CBoW और स्किप-ग्राम एल्गोरिदम को शब्दों को वेक्टर में बदलने के लिए दिखाने वाली छवि।](../../../../../lessons/5-NLP/14-Embeddings/images/example-algorithms-for-converting-words-to-vectors.png)

Google News डेटासेट पर प्री-ट्रेन किए गए वर्ड2वेक एम्बेडिंग के साथ प्रयोग करने के लिए, हम **gensim** लाइब्रेरी का उपयोग कर सकते हैं। नीचे हम 'neural' के सबसे समान शब्दों को ढूंढते हैं।

> **Note:** जब आप पहली बार शब्द वेक्टर बनाते हैं, तो उन्हें डाउनलोड करने में कुछ समय लग सकता है!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


हम शब्द से वेक्टर एम्बेडिंग भी गणना कर सकते हैं, जिसे वर्गीकरण मॉडल के प्रशिक्षण में उपयोग किया जा सकता है (स्पष्टता के लिए हम केवल वेक्टर के पहले 20 घटक दिखाते हैं):


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

CBoW और Skip-Grams दोनों "predictive" embeddings हैं, क्योंकि ये केवल स्थानीय संदर्भों को ध्यान में रखते हैं। Word2Vec वैश्विक संदर्भ का लाभ नहीं उठाता है।

**FastText**, Word2Vec पर आधारित है और प्रत्येक शब्द और उसमें पाए जाने वाले अक्षर n-grams के लिए वेक्टर प्रतिनिधित्व सीखता है। इन प्रतिनिधित्वों के मानों को प्रत्येक प्रशिक्षण चरण में एक वेक्टर में औसत किया जाता है। हालांकि यह प्री-ट्रेनिंग में काफी अतिरिक्त गणना जोड़ता है, लेकिन यह वर्ड एम्बेडिंग्स को सब-वर्ड जानकारी को एन्कोड करने में सक्षम बनाता है।

एक और विधि, **GloVe**, सह-अस्तित्व मैट्रिक्स (co-occurrence matrix) के विचार का उपयोग करती है और सह-अस्तित्व मैट्रिक्स को अधिक अभिव्यक्तिपूर्ण और गैर-रेखीय (non-linear) वर्ड वेक्टर में विभाजित करने के लिए न्यूरल विधियों का उपयोग करती है।

आप उदाहरण के साथ खेल सकते हैं और embeddings को FastText और GloVe में बदल सकते हैं, क्योंकि gensim कई अलग-अलग वर्ड एम्बेडिंग मॉडल का समर्थन करता है।


## PyTorch में प्री-ट्रेंड एम्बेडिंग्स का उपयोग करना

हम ऊपर दिए गए उदाहरण को संशोधित कर सकते हैं ताकि हमारे एम्बेडिंग लेयर के मैट्रिक्स को पहले से तैयार किए गए सेमांटिकल एम्बेडिंग्स, जैसे Word2Vec, से प्री-पॉप्युलेट किया जा सके। हमें यह ध्यान रखना होगा कि प्री-ट्रेंड एम्बेडिंग और हमारे टेक्स्ट कॉर्पस की वोकैब्युलरी शायद मेल नहीं खाएगी, इसलिए हम उन शब्दों के लिए वेट्स को रैंडम वैल्यू से इनिशियलाइज़ करेंगे जो गायब हैं:


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

हमारे मामले में, हमें सटीकता में बहुत अधिक वृद्धि नहीं दिखती है, जो संभवतः अलग-अलग शब्दावली के कारण है।  
अलग-अलग शब्दावली की समस्या को हल करने के लिए, हम निम्नलिखित समाधानों में से एक का उपयोग कर सकते हैं:  
* हमारे शब्दावली पर word2vec मॉडल को पुनः प्रशिक्षित करें  
* प्री-ट्रेंड word2vec मॉडल की शब्दावली के साथ हमारा डेटासेट लोड करें। डेटासेट को लोड करने के लिए उपयोग की जाने वाली शब्दावली को लोडिंग के दौरान निर्दिष्ट किया जा सकता है।  

दूसरा तरीका अधिक आसान लगता है, खासकर क्योंकि PyTorch `torchtext` फ्रेमवर्क में एम्बेडिंग के लिए बिल्ट-इन सपोर्ट है। उदाहरण के लिए, हम निम्नलिखित तरीके से GloVe-आधारित शब्दावली को इंस्टैंशिएट कर सकते हैं:  


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


लोडेड शब्दावली में निम्नलिखित बुनियादी ऑपरेशन्स होते हैं:
* `vocab.stoi` डिक्शनरी हमें किसी शब्द को उसके डिक्शनरी इंडेक्स में बदलने की अनुमति देती है
* `vocab.itos` इसका उल्टा करता है - नंबर को शब्द में बदलता है
* `vocab.vectors` एम्बेडिंग वेक्टर का ऐरे है, इसलिए किसी शब्द `s` की एम्बेडिंग प्राप्त करने के लिए हमें `vocab.vectors[vocab.stoi[s]]` का उपयोग करना होगा

यहाँ एम्बेडिंग्स के साथ छेड़छाड़ का एक उदाहरण है, जो समीकरण **kind-man+woman = queen** को प्रदर्शित करता है (मुझे इसे काम करने के लिए कोएफिशिएंट को थोड़ा समायोजित करना पड़ा):


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

उन एम्बेडिंग्स का उपयोग करके वर्गीकरणकर्ता को प्रशिक्षित करने के लिए, हमें पहले अपने डेटासेट को GloVe शब्दावली का उपयोग करके एन्कोड करना होगा:


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

जैसा कि हमने ऊपर देखा, सभी वेक्टर एम्बेडिंग्स `vocab.vectors` मैट्रिक्स में संग्रहीत होती हैं। इसे एम्बेडिंग लेयर के वेट्स में सरल कॉपीिंग के माध्यम से लोड करना बहुत आसान बनाता है:


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

हमारी सटीकता में महत्वपूर्ण वृद्धि न देखने के कारणों में से एक यह है कि हमारे डेटासेट के कुछ शब्द प्री-ट्रेंड GloVe शब्दावली में नहीं हैं, और इसलिए उन्हें अनदेखा कर दिया जाता है। इस तथ्य को दूर करने के लिए, हम अपने डेटासेट पर अपने स्वयं के एम्बेडिंग्स को प्रशिक्षित कर सकते हैं।


## संदर्भात्मक एम्बेडिंग्स

पारंपरिक प्रीट्रेंड एम्बेडिंग प्रतिनिधित्व, जैसे Word2Vec, की एक मुख्य सीमा शब्दार्थ अस्पष्टता (word sense disambiguation) की समस्या है। जबकि प्रीट्रेंड एम्बेडिंग्स शब्दों के कुछ अर्थ को संदर्भ में पकड़ सकते हैं, एक शब्द के हर संभावित अर्थ को एक ही एम्बेडिंग में एन्कोड किया जाता है। यह डाउनस्ट्रीम मॉडल्स में समस्याएं पैदा कर सकता है, क्योंकि कई शब्दों, जैसे 'play', के अलग-अलग संदर्भों में अलग-अलग अर्थ हो सकते हैं।

उदाहरण के लिए, 'play' शब्द का इन दो वाक्यों में काफी अलग अर्थ है:
- मैं थिएटर में एक **play** देखने गया।
- जॉन अपने दोस्तों के साथ **play** करना चाहता है।

ऊपर दिए गए प्रीट्रेंड एम्बेडिंग्स 'play' शब्द के इन दोनों अर्थों को एक ही एम्बेडिंग में दर्शाते हैं। इस सीमा को दूर करने के लिए, हमें **भाषा मॉडल** पर आधारित एम्बेडिंग्स बनानी होंगी, जो एक बड़े टेक्स्ट कॉर्पस पर प्रशिक्षित होता है और *जानता है* कि शब्दों को विभिन्न संदर्भों में कैसे जोड़ा जा सकता है। संदर्भात्मक एम्बेडिंग्स पर चर्चा करना इस ट्यूटोरियल के दायरे से बाहर है, लेकिन हम अगले यूनिट में भाषा मॉडलों पर बात करते समय इस पर वापस आएंगे।



---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता सुनिश्चित करने का प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को प्रामाणिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम उत्तरदायी नहीं हैं।
